# Facebook Ads Weekly Report

## Import data and update some values manually

In [6]:
import pandas as pd

# read file
df = pd.read_excel('fb_raw_data.xlsx')

# Need to be updated
Days_lastmonth =  30
Days_analysed = 15
Thismonth = 'Oct 1-15'
Lastmonth = 'Sep Prorated'

## Classify the campaigns into countries

In [7]:
# COV campaigns
Cov = df[df['Campaign'].str.contains('Cov')]
# Awa camapigns
Awa = df[df['Campaign'].str.contains('Awa')]

# 简化所有campaign的名字
# Function to extract and combine values
def generate_new_campaign_name(campaign_name):
    part1 = campaign_name.split('_')[1]  # Split by underscores, take the second part
    part2 = '-'.join(campaign_name.split('-')[1:])   # Split by '-' and take the second part
    new_campaign_name = f"{part1}-{part2}"  # Combine the parts with '-'
    return new_campaign_name
# Apply the function to the 'Campaign' column
Cov.loc[:,'Campaign'] = Cov['Campaign'].apply(generate_new_campaign_name)
Awa.loc[:,'Campaign'] = Awa['Campaign'].apply(generate_new_campaign_name)

# 之后只对Cov进行操作，如果后续需要Awa,需要自行加上
# Create df_sg by selecting rows where 'campaign' column contains 'SG'
SG = Cov[Cov['Campaign'].str.contains('SG')]
# Create df_uk by selecting rows where 'campaign' column contains 'UK'
UK = Cov[Cov['Campaign'].str.contains('UK')]
# Create df_my by selecting rows where 'campaign' column contains 'MY'
MY = Cov[Cov['Campaign'].str.contains('MY')]

Cov

,Campaign,Ad_set,Ad,Currency,Cost,Cost_lm,Amount spent (USD) Change,Amount spent (USD) Change (%),Impression,Impression_lm,...,Purchase_lm,Purchases Change,Purchases Change (%),Revenue,Revenue_lm,Purchases conversion value Change,Purchases conversion value Change (%),Ad delivery,Reporting starts,Reporting ends
0,SG-Retargeting,IG Shopping,Vid30_psvoiceover-spg,USD,829.84,NaN,NaN,NaN,56638.0,NaN,...,NaN,NaN,NaN,1659.83,NaN,NaN,NaN,active,2023-10-01,2023-10-15
1,SG-Prop-SPG,LAL3_Purchase2023,Vid30_psvoiceover-spg,USD,475.82,82.95,392.87,473.622664,53509.0,8647.0,...,NaN,NaN,NaN,431.66,NaN,NaN,NaN,active,2023-10-01,2023-10-15
2,SG-Retargeting,IG Shopping,Static_BMI-SG-app(smallpic),USD,645.25,248.63,396.62,159.522182,53255.0,19547.0,...,4.0,13.0,325.000000,1978.31,544.31,1434.00,263.452812,active,2023-10-01,2023-10-15
3,MY-Prop,LAL1-MYwaitlist,Static_launch-branded,USD,170.04,385.64,-215.60,-55.907064,51632.0,87366.0,...,7.0,-5.0,-71.428571,119.67,365.62,-245.95,-67.269296,inactive,2023-10-01,2023-10-15
4,SG-Gen,Lookalike_LAL3 Quiz Completion,Static_direct_lube,USD,266.63,786.03,-519.40,-66.078903,41785.0,105225.0,...,15.0,-12.0,-80.000000,79.97,934.04,-854.07,-91.438268,inactive,2023-10-01,2023-10-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,UK-Prop-AIvoice,Broad_P30-50,Vid30_Inclusivity_uk-app(voice),USD,NaN,0.06,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive,NaN,NaN
480,MY-Retargeting,All Visitors - 90D,Static_betweenIVFIUI-spg,USD,NaN,0.04,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,archived,NaN,NaN
481,UK-Prop,Broad_P30-50,Vid_helpcreatefamily_uk-app(voice),USD,NaN,0.03,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive,NaN,NaN
482,SG-Prop-AIvoice,Lookalike_LAL1 Modest,Vid30_Inclusivity_sg-app(no voice),USD,NaN,0.04,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inactive,NaN,NaN


In [8]:
# First column name in the Ads analysis
first_column_name = Thismonth + ' Ads'

# Campaign orders, Need to be updated if new campaign is created

columns_order_ad = ['Amount_spent', 'Impressions', 'Link_clicks', 'Adds_to_cart','Purchases', 'Revenue', 'CTR(%)','ROAS', 'CPA', 'CR(%)']

## Country overall performance

In [9]:
# Function to calculate overall performance
def calculate_overall_performance(df, days_analysed, days_lastmonth, df_name):
    overall_thismonth = df[['Cost', 'Impression', 'Click', 'ATC', 'Purchase', 'Revenue']].sum()
    overall_thismonth['ROAS'] = overall_thismonth['Revenue'] / overall_thismonth['Cost']
    overall_thismonth['CPA'] = overall_thismonth['Cost'] / overall_thismonth['Purchase']
    overall_thismonth['CR(%)'] = overall_thismonth['Purchase'] / overall_thismonth['Click']

    overall_lastmonth = df[['Cost_lm', 'Impression_lm', 'Click_lm', 'ATC_lm', 'Purchase_lm', 'Revenue_lm']].sum()
    overall_lastmonth = overall_lastmonth / days_lastmonth * days_analysed
    overall_lastmonth['ROAS'] = overall_lastmonth['Revenue_lm'] / overall_lastmonth['Cost_lm']
    overall_lastmonth['CPA'] = overall_lastmonth['Cost_lm'] / overall_lastmonth['Purchase_lm']
    overall_lastmonth['CR(%)'] = overall_lastmonth['Purchase_lm'] / overall_lastmonth['Click_lm']

    overall_lastmonth.index = overall_thismonth.index
    overall = pd.concat([overall_thismonth, overall_lastmonth], axis=1)
    overall = overall.reset_index()
    overall.columns = [df_name, Thismonth, Lastmonth]

    overall['Growth(%)'] = overall[Thismonth] / overall[Lastmonth] - 1

    return overall

# Call the function for each DataFrame
overall_SG = calculate_overall_performance(SG, Days_analysed, Days_lastmonth, 'SG')
overall_UK = calculate_overall_performance(UK, Days_analysed, Days_lastmonth, 'UK')
overall_MY = calculate_overall_performance(MY, Days_analysed, Days_lastmonth, 'MY')

## Country Campaign Breakdown

In [10]:
def calculate_campaign_performance(df, days_analysed, days_lastmonth, df_name):
    # This month performance
    this_month = pd.pivot_table(df, index=['Campaign'], values=['Cost', 'Impression', 'Click', 'ATC', 'Purchase', 'Revenue'], aggfunc='sum')
    this_month['ROAS'] = this_month['Revenue'] / this_month['Cost']
    this_month['CPA'] = this_month['Cost'] / this_month['Purchase']
    this_month['CR(%)'] = this_month['Purchase'] / this_month['Click']
    columns_order_thismonth = ['Cost', 'Impression', 'Click', 'ATC', 'Purchase', 'Revenue', 'ROAS', 'CPA', 'CR(%)']
    this_month = this_month.reindex(columns=columns_order_thismonth)

    # Last month performance
    last_month = pd.pivot_table(df, index=['Campaign'], values=['Cost_lm', 'Impression_lm', 'Click_lm', 'ATC_lm', 'Purchase_lm', 'Revenue_lm'], aggfunc='sum')
    last_month = last_month / days_lastmonth * days_analysed
    last_month['ROAS'] = last_month['Revenue_lm'] / last_month['Cost_lm']
    last_month['CPA'] = last_month['Cost_lm'] / last_month['Purchase_lm']
    last_month['CR(%)'] = last_month['Purchase_lm'] / last_month['Click_lm']
    columns_order_lastmonth = ['Cost_lm', 'Impression_lm', 'Click_lm', 'ATC_lm','Purchase_lm', 'Revenue_lm', 'ROAS', 'CPA', 'CR(%)']
    last_month = last_month.reindex(columns=columns_order_lastmonth)

    # Combine two DataFrames
    last_month.columns = this_month.columns
    campaign_performance = pd.concat([this_month, last_month], axis=0)

    # Calculate growth
    growth = this_month / last_month - 1
    # Combine series
    campaign_performance = pd.concat([campaign_performance, growth],axis=0)
    # Transpose and reset index
    campaign_performance = campaign_performance.transpose().reset_index()

    # Rename columns
    campaign_name = campaign_performance.columns
    number = int((campaign_performance.shape[1] - 1) / 3)
    column_name = [df_name] + [Thismonth] * number + [Lastmonth] * number + ['Growth(%)'] * number
    campaign_performance = pd.DataFrame([campaign_name] + campaign_performance.values.tolist(), columns=column_name)

    return campaign_performance

# Call the function for each DataFrame
campaign_SG = calculate_campaign_performance(SG, Days_analysed, Days_lastmonth, 'Campaign_SG')
campaign_UK = calculate_campaign_performance(UK, Days_analysed, Days_lastmonth, 'Campaign_UK')
campaign_MY = calculate_campaign_performance(MY, Days_analysed, Days_lastmonth, 'Campaign_MY')

# Print the results
print(campaign_SG)
print(campaign_UK)
print(campaign_MY)

  Campaign_SG  Oct 1-15    Oct 1-15        Oct 1-15          Oct 1-15  \
0       index    SG-Gen  SG-Gen-OPR  SG-MFF (Purch)  SG-NewAdsTesting   
1        Cost    1267.0         0.0         1107.88           1371.16   
2  Impression  187335.0         0.0        152797.0          117488.0   
3       Click     424.0         0.0           501.0             717.0   
4         ATC      23.0         0.0            22.0              60.0   
5    Purchase       7.0         0.0            12.0              14.0   
6     Revenue    349.45         0.0         1224.98           1032.94   
7        ROAS  0.275809         NaN        1.105697          0.753333   
8         CPA     181.0         NaN       92.323333             97.94   
9       CR(%)  0.016509         NaN        0.023952          0.019526   

                  Oct 1-15               Oct 1-15         Oct 1-15  \
0  SG-Prop-5MonthFertility  SG-Prop-5mFP(Testing)  SG-Prop-AIvoice   
1                      0.0                 632.87       

## Breakdown into ad set for each campaign

In [11]:
# 获取所有唯一的广告活动名称
unique_campaigns = Cov['Campaign'].unique()

# 存储所有结果的列表
results = []
for campaign in unique_campaigns:
    # 获取特定广告活动的子数据集
    campaign_data = Cov[Cov['Campaign'] == campaign]
    results.append(campaign_data)

print(results)

[           Campaign            Ad_set  \
0    SG-Retargeting       IG Shopping   
2    SG-Retargeting       IG Shopping   
14   SG-Retargeting       IG Shopping   
26   SG-Retargeting     APP PDP - 90D   
27   SG-Retargeting     SPG PDP - 90D   
..              ...               ...   
439  SG-Retargeting     SPG PDP - 90D   
448  SG-Retargeting     APP PDP - 90D   
454  SG-Retargeting       IG Shopping   
478  SG-Retargeting  HMTSCT PDP - 90D   
483  SG-Retargeting       IG Shopping   

                                               Ad Currency    Cost  Cost_lm  \
0                           Vid30_psvoiceover-spg      USD  829.84      NaN   
2                     Static_BMI-SG-app(smallpic)      USD  645.25   248.63   
14                          Vid30_reaction-app(V)      USD  304.16    93.58   
26                          Vid30_reaction-app(V)      USD  284.14    51.41   
27   Static_testimonial-saveeverysperm-spg (2mod)      USD  282.71    35.60   
..                              

In [12]:
# 获取所有唯一的广告活动名称
unique_campaigns = Cov['Campaign'].unique()

# 存储所有结果的列表
results = []

# 循环遍历每个唯一的广告活动
for campaign in unique_campaigns:
    # 获取特定广告活动的子数据集
    campaign_raw = Cov[Cov['Campaign'] == campaign]
    
    # 进行所需的操作，这里以你提供的示例操作为准
    # 例如，你可以使用 campaign_data 来执行和存储 campaign_data, campaign_data_lm, 和 Growth 的计算
    
   ## SG Retargeting Campaign this month
    campaign_data = pd.pivot_table(campaign_raw, index=['Ad_set'], values=['Cost', 'Impression', 'Click', 'ATC', 'Purchase', 'Revenue'], aggfunc='sum')
    campaign_data['ROAS'] = campaign_data['Revenue'] / campaign_data['Cost']
    campaign_data['CPA'] = campaign_data['Cost'] / campaign_data['Purchase']
    campaign_data['CR(%)'] = campaign_data['Purchase'] / campaign_data['Click']
    # Change order
    columns_order_thismonth = ['Cost', 'Impression', 'Click', 'ATC', 'Purchase', 'Revenue', 'ROAS', 'CPA', 'CR(%)']
    campaign_data = campaign_data.reindex(columns=columns_order_thismonth)


    # SG Pro rata performance last month
    campaign_data_lm = pd.pivot_table(campaign_raw, index=['Ad_set'], values=['Cost_lm', 'Impression_lm', 'Click_lm', 'ATC_lm', 'Purchase_lm', 'Revenue_lm'], aggfunc='sum')
    # Calculate the pro rata
    campaign_data_lm = campaign_data_lm / Days_lastmonth * Days_analysed
    # Calculate the ROAS, CPA and CR
    campaign_data_lm['ROAS'] = campaign_data_lm['Revenue_lm'] / campaign_data_lm['Cost_lm']
    campaign_data_lm['CPA'] = campaign_data_lm['Cost_lm'] / campaign_data_lm['Purchase_lm']
    campaign_data_lm['CR(%)'] = campaign_data_lm['Purchase_lm'] / campaign_data_lm['Click_lm']
    # Change order
    columns_order_lastmonth = ['Cost_lm', 'Impression_lm', 'Click_lm', 'ATC_lm','Purchase_lm', 'Revenue_lm', 'ROAS', 'CPA', 'CR(%)']
    campaign_data_lm = campaign_data_lm.reindex(columns=columns_order_lastmonth)


    # Combine two series
    # Change the series into same index, in order to concat
    campaign_data_lm.columns = campaign_data.columns
    campaign_data_result = pd.concat([campaign_data, campaign_data_lm],axis=0)


    # Calculate growth
    Growth = campaign_data  / campaign_data_lm - 1
    # Combine series
    campaign_data_result = pd.concat([campaign_data_result, Growth],axis=0)
    # Transpose and reset index
    campaign_data_result = campaign_data_result.transpose().reset_index()
    
    # Change columns name
    Campaign_name = campaign_data_result.columns
    Number = int((campaign_data_result.shape[1] - 1) / 3)
    Column_name = [campaign] + [Thismonth] * Number + [Lastmonth] * Number +  ['Growth(%)'] * Number
    campaign_data_result = pd.DataFrame([Campaign_name] + campaign_data_result.values.tolist(), columns = Column_name)

    
    results.append(campaign_data_result)


## Export the data

In [13]:
import pandas as pd

# 假设你有一些名为 overall_SG、overall_UK、overall_MY、campaign_SG、campaign_UK、campaign_MY 和 results 的 DataFrame

dfs = [overall_SG, overall_UK, overall_MY, campaign_SG, campaign_UK, campaign_MY]  # 不包括 results

writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
sheet_name = 'Sheet1'
start_row = 0

for df in dfs:
    df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=start_row)
    start_row += df.shape[0] + 2

# 处理 results 列表中的结果
for result_df in results:
    result_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=start_row)
    start_row += result_df.shape[0] + 2

writer.save()